In [1]:
(ns ground-up.ch5)

nil

### Macroexpansion

In [2]:
(defmacro ignore
    "Cancels the evaluation of an expression, returning nil instead"
    [expr]
    nil)

#'ground-up.ch5/ignore

In [3]:
(ignore (+ 1 2)) ;; (+ 1 2) was never evaluated at all

nil

In [4]:
(def x 1)

#'ground-up.ch5/x

In [5]:
(ignore (def x 2))

nil

In [6]:
x

1

- during macroexpansion time, the expression is replaced by the expression `nil`, which is then evaluated to `nil`
- __Where functions rewrite values, macros rewrite code__

In [7]:
(defmacro rev [fun & args]
    (cons fun (reverse args)))

#'ground-up.ch5/rev

In [8]:
;; takes an expression and returns that expression with all macros expanded
(macroexpand '(rev str "hi" (+ 1 2)))

(str (+ 1 2) "hi")

In [9]:
;; takes an expression and evaluates it. 
(eval (macroexpand '(rev str "hi" (+ 1 2))))

"3hi"

In [10]:
(eval (rev str "hi" (+ 1 2))) ;; so `eval` first expand macro and evaluates?

"3hi"

- Clojure macroexpands, then evaluates.
- The first stage transforms code, the second stage transforms values

### Defining new syntax
- What kind of transformations are best expressed with macros?

In [11]:
(clojure.repl/source or)

(defmacro or
  "Evaluates exprs one at a time, from left to right. If a form
  returns a logical true value, or returns that value and doesn't
  evaluate any of the other expressions, otherwise it returns the
  value of the last expression. (or) returns nil."
  {:added "1.0"}
  ([] nil)
  ([x] x)
  ([x & next]
      `(let [or# ~x]
         (if or# or# (or ~@next)))))


nil

- `` ` `` syntax-quote: it is like regular quote - preventing evaluation of the following list, but we can escape the quoting rule and substitute in regularly evaluated expressions using unquote(`~`) and unquote-splice (`~@`)
- think of a syntax-quoted expression liek a template for code, with some parts filled in by evaluated forms

In [12]:
(let [x 2] `(inc x))

(clojure.core/inc ground-up.ch5/x)

In [13]:
(let [x 2] `(inc ~x))

(clojure.core/inc 2)

In [14]:
`(foo ~[1 2 3])

(ground-up.ch5/foo [1 2 3])

In [15]:
`(foo ~@[1 2 3])

(ground-up.ch5/foo 1 2 3)

In [16]:
`(foo ~@'(1 2 3))

(ground-up.ch5/foo 1 2 3)

In [17]:
(clojure.pprint/pprint (macroexpand '(or a b c d)))

(let*
 [or__5516__auto__ a]
 (if or__5516__auto__ or__5516__auto__ (clojure.core/or b c d)))


nil

- whenever we need a new variable in a macro, we use `gensym` to generate a new symbol
- tack on a `#` to the end of a symbol in a syntax-quoted expression, it'll be expanded to a particular gensym
- to escape the safety feature and override local variables, use `~'foo` instead of `foo#` to do symbol capture

In [18]:
(gensym "hi")

hi4142

In [19]:
(gensym "hi")

hi4145

In [20]:
(gensym "hi")

hi4148

In [21]:
`(let [x# 2] x#)

(clojure.core/let [x__4150__auto__ 2] x__4150__auto__)

In [22]:
(clojure.pprint/pprint (clojure.walk/macroexpand-all
                         '(or (mossy? stone) (cool? stone) (wet? stone))))

(let*
 [or__5516__auto__ (mossy? stone)]
 (if
  or__5516__auto__
  or__5516__auto__
  (let*
   [or__5516__auto__ (cool? stone)]
   (if or__5516__auto__ or__5516__auto__ (wet? stone)))))


nil

### Control flow

In [23]:
;; `do` evaluate more than one expression in order
(if (pos? -5)
    (prn "-5 is positive")
    (do
        (prn "-5 is negative")
        (prn "Who would have thought?")))

"-5 is negative"
"Who would have thought?"


nil

- `prn` is a function which has a side effect. it prints a message to the screen and returns `nil`
- `if` only takes a single expression per branch
- `do` wraps up several expressions into a single expression, and evaluate them in order
- `do` returns the value of the final expression
- use `when` if you want to take only one branch of an `if`
- `when` takes any number of expressions!

In [24]:
(when false
    (prn :hi)
    (prn :there))

nil

In [25]:
(when true
    (prn :hi)
    (prn :there))

:hi
:there


nil

In [26]:
(when-not (number? "a string")
    :here)

:here

In [27]:
(if-not (vector? (list 1 2 3))
    :a
    :b)

:a

In [28]:
;; reuse the result of some operation
(when-let [x (+ 1 2 3 4)]
    (str x))

"10"

- `while` evaluates an expression so long as its predicate is truthy. 
- This is generally useful only for side effects, like `prn` or `def`; things that change the state of the world

In [29]:
(def x 0)
(while (< x 5)
    (prn x)
    (def x (inc x)))

0
1
2
3
4


nil

In [30]:
;; takes any number of test/expression pairs
;; tries each test in turn
;; the first test which evaluates truthy causes the following expression to be evaluated
;; returns that expression's value
(cond
    (= 2 5) :nope
    (= 3 3) :yep
    (= 5 5) :cant-get-here
    :else   :a-default-value)

:yep

In [31]:
(defn category
    "Determine the Saffir-Simpson category of a hurricane, by wind speed in meters/sec"
    [wind-speed]
    (condp <= wind-speed
        70 :F5
        60 :F4
        49 :F3
        42 :F2
           :F1))  ;; Default value

#'ground-up.ch5/category

In [32]:
(category 10)

:F1

In [33]:
(category 50)

:F3

In [34]:
(category 100)

:F5

In [35]:
(defn with-tax
    "Computes the total cost, with tax, of a purchase in the given state."
    [state subtotal]
    (case state
        :WA (* 1.065 subtotal)
        :OR subtotal
        :CA (* 1.075 subtotal)
        subtotal))

#'ground-up.ch5/with-tax

- Unlike `cond` and `condp`, `case` does not evaluate its tests in order. It jumps immediately ot the matching expression. This makes `case` much faster when there are many branches to take, at the cost of reduced generality. 

### Recursion

In [36]:
;; functions call themselves explicitly
(defn sum [numbers]
    (if-let [n (first numbers)]
        (+ n (sum (rest numbers)))
        0))

#'ground-up.ch5/sum

In [37]:
(sum (range 10))

45

In [38]:
(sum (range 100000))

Execution error (StackOverflowError) at ground-up.ch5/sum (REPL:3).
null


class java.lang.StackOverflowError: 

- in order to add `n` and `(sum (rest numbers))`, we have to call `sum` first - while holding onto the memory for `n` and `numbers`. We can't reuse that memory until every single recursive call has completed. 

In [39]:
;; a variation
(defn sum
    ([numbers]
     (sum 0 numbers))
    ([subtotal numbers]
     (if-let [n (first numbers)]
         (recur (+ subtotal n) (rest numbers))
         subtotal)))

#'ground-up.ch5/sum

In [40]:
(sum (range 100000))

4999950000

- in its two-arguent form, `sum` now takes an accumulator, `subtotal`, which represents the count so far
- `recur` has take the palce of `sum`
- the final expression to be evaluated is not `+`, but `sum` itself. 
- we don't need to hang on to any of the variables in this function any more, because the final return value won't depend on them. 
- `recur` hints to the Clojure compiler that we don't need to hold on to the stack, and can re-use that space for other things. 
- This is called a _tail-recursive_ function, and it requires only a single stack frame no matter how deep the recursive calls go. 
use `recur` wherever possible!

In [41]:
;; use recur within the context of the `loop` macro
;; think of loop as a recursive let
(loop [i 0
       nums []]
    (if (< 10 i)
        nums
        (recur (inc i) (conj nums i))))

[0 1 2 3 4 5 6 7 8 9 10]

### Laziness
- most of the sequences returned by `map`, `filter`, `iterate`, `repeatedly` and so on, were _lazy_
- This Laziness is provided by a macro `lazy-seq`

In [42]:
(clojure.repl/source lazy-seq)

(defmacro lazy-seq
  "Takes a body of expressions that returns an ISeq or nil, and yields
  a Seqable object that will invoke the body only the first time seq
  is called, and will cache the result and return it on all subsequent
  seq calls. See also - realized?"
  {:added "1.0"}
  [& body]
  (list 'new 'clojure.lang.LazySeq (list* '^{:once true} fn* [] body)))


nil

In [43]:
(clojure.repl/source filter)

(defn filter
  "Returns a lazy sequence of the items in coll for which
  (pred item) returns logical true. pred must be free of side-effects.
  Returns a transducer when no collection is provided."
  {:added "1.0"
   :static true}
  ([pred]
    (fn [rf]
      (fn
        ([] (rf))
        ([result] (rf result))
        ([result input]
           (if (pred input)
             (rf result input)
             result)))))
  ([pred coll]
   (lazy-seq
    (when-let [s (seq coll)]
      (if (chunked-seq? s)
        (let [c (chunk-first s)
              size (count c)
              b (chunk-buffer size)]
          (dotimes [i size]
              (let [v (.nth c i)]
                (when (pred v)
                  (chunk-append b v))))
          (chunk-cons (chunk b) (filter pred (chunk-rest s))))
        (let [f (first s) r (rest s)]
          (if (pred f)
            (cons f (filter pred r))
            (filter pred r))))))))


nil

In [44]:
(defn integers
    [x]
    (lazy-seq
     (cons x (integers (inc x)))))

#'ground-up.ch5/integers

- This is infinitely recursive
- `lazy-seq` interrupted that recursion and restructured it into a sequence which constructs elements ony when they are requested

In [45]:
(def xs (integers 0))

#'ground-up.ch5/xs

In [46]:
(take 10 xs)

(0 1 2 3 4 5 6 7 8 9)

In [47]:
(take 100000000 xs) ;; why not stackoverflow????

Error printing return value (OutOfMemoryError) at java.util.Arrays/copyOf (Arrays.java:3332).
Java heap space


class clojure.lang.ExceptionInfo: 

In [48]:
(def x (delay
        (prn "computing a really big number")
        (last (take 100000000 (iterate inc 0)))))

#'ground-up.ch5/x

In [49]:
x

#delay[{:status :pending, :val nil} 0x15da4b2b]

In [50]:
(deref x) ;; why not throw a out of memory error???

"computing a really big number"


99999999

### List comprehensions
- the list comprehension macro - `for` - combines recursion and laziness. 
- `for` takes a vector of bindings, like `let`
- `for` binds its variables to each possible combination of elemnets in their corresponding sequences. 

In [51]:
;; this simplest form works like `map`
(for [x (range 10)] (- x))

(0 -1 -2 -3 -4 -5 -6 -7 -8 -9)

In [52]:
;; for each x in sequence [1 2 3], and for each y in the sequence [:a :b], find all [x y] pairs
(for [x [1 2 3]
      y [:a :b]]
    [x y])

([1 :a] [1 :b] [2 :a] [2 :b] [3 :a] [3 :b])

In [53]:
(for [x      (range 5)
      y      (range 5)
      :when  (and (even? x) (odd? y))]
    [x y])

([0 1] [0 3] [2 1] [2 3] [4 1] [4 3])

In [54]:
(for [x      (range 5)
      y      (range 5)
      :while  (and (> 3 x) (> 1 y))]
    [x y])

([0 0] [1 0] [2 0])

### The threading macros
- `->>` takes a previous expression and insert it at the end of the next expression
- `->` takes a previous expression and insert it as the first argument of the next expression

In [55]:
(->>
    (range 10)
    (filter odd?)
    (reduce +))

25

In [56]:
(clojure.walk/macroexpand-all 
 '(->>
    (range 10)
    (filter odd?)
    (reduce +)))

(reduce + (filter odd? (range 10)))

In [57]:
(-> {:a :b}
    (assoc :apple :pear)
    (assoc :phone :laptop))

{:a :b, :apple :pear, :phone :laptop}

In [58]:
(clojure.walk/macroexpand-all 
 '(-> {:a :b}
    (assoc :apple :pear)
    (assoc :phone :laptop)))

(assoc (assoc {:a :b} :apple :pear) :phone :laptop)

### Problems

In [59]:
;; 2. use threading macro to find how many numbers from 0-9999 are palindromes. 
(-> (fn [x] (= (seq (str x)) (reverse (str x))))
    (filter (range 9999))
    (count))

198

In [60]:
;; 3. write a macro `id`
(defmacro id
    [fun & args]
    (cons fun args))

#'ground-up.ch5/id

In [61]:
(id str "a" "b" 111)

"ab111"

In [62]:
;; 4. write a macro `log`
(def logging-enabled true)
(defmacro log
    [expr]
    (if logging-enabled
        (prn expr)
        nil))

#'ground-up.ch5/log

In [63]:
(log :hi)

:hi


nil

In [64]:
(def logging-enabled false)

#'ground-up.ch5/logging-enabled

In [65]:
(log :hi)

nil

- I guess we want to do this check during compilation since we can use a global var to turn logging on and off throughout the program. 
- Is the downside being that we do need to define the varible `logging-enabled` before defining the macro??

In [66]:
;; 5 define a macro that return the ratio
(defmacro exact
    [expr]
    (let [fun (first expr)
          args (rest expr)]
    `(~fun ~@(map rationalize args))))

#'ground-up.ch5/exact

In [67]:
(exact (* 2.45 100))

245N